# Importation des données de Crimes.xlsx vers SQLite
Ce notebook charge un fichier Excel et insère ses données dans une base SQLite.

In [10]:

import sqlite3
import pandas as pd
import openpyxl

### Lecture du fichier Excel

In [11]:
file_path = r"C:\Users\JadSamain\OneDrive - ISTEP\Bureau\IUT\SAE_NoSQL\SAE_MigrationNoSQL\CSV_Modifies\Crimes.xlsx"
xls = pd.ExcelFile(file_path)

### Connexion à la base SQLite et création des tables

In [12]:
# Connexion à la base SQLite
db_path = r"C:\Users\JadSamain\OneDrive - ISTEP\Bureau\IUT\SAE_NoSQL\SAE_MigrationNoSQL\BD\bdd"
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# Création des tables
cursor.executescript('''
CREATE TABLE IF NOT EXISTS Departement (
    id_dep TEXT PRIMARY KEY,
    lib_dep TEXT
);

CREATE TABLE IF NOT EXISTS Brigade (
    id_brigade INTEGER PRIMARY KEY AUTOINCREMENT,
    lib_brigade TEXT,
    is_pn BOOLEAN,
    id_dep TEXT,
    FOREIGN KEY (id_dep) REFERENCES Departement(id_dep)
);

CREATE TABLE IF NOT EXISTS Crime (
    code_index INTEGER PRIMARY KEY,
    lib_index TEXT
);

CREATE TABLE IF NOT EXISTS Enregistrer (
    id_brigade INTEGER,
    id_crime INTEGER,
    annee INTEGER,
    PRIMARY KEY (id_brigade, id_crime, annee),
    FOREIGN KEY (id_brigade) REFERENCES Brigade(id_brigade),
    FOREIGN KEY (id_crime) REFERENCES Crime(code_index)
);

CREATE TABLE IF NOT EXISTS PN (
    id_pn INTEGER PRIMARY KEY AUTOINCREMENT,
    lib_pn TEXT,
    perimetre TEXT,
    id_brigade INTEGER,
    FOREIGN KEY (id_brigade) REFERENCES Brigade(id_brigade)
);

CREATE TABLE IF NOT EXISTS GN (
    id_gn INTEGER PRIMARY KEY AUTOINCREMENT,
    lib_gn TEXT,
    id_brigade INTEGER,
    FOREIGN KEY (id_brigade) REFERENCES Brigade(id_brigade)
);
''')

print("Tables créées avec succès.")

Tables créées avec succès.


In [13]:
# Insertion des données
for sheet_name in xls.sheet_names:
    df = xls.parse(sheet_name)
    df = df.iloc[2:]  # Suppression des en-têtes inutiles
    df = df.reset_index(drop=True)
    
    for i, row in df.iterrows():
        crime_index = row.iloc[0]
        crime_lib = row.iloc[1]
        cursor.execute("INSERT OR IGNORE INTO Crime (code_index, lib_index) VALUES (?, ?)", (crime_index, crime_lib))
        
        for col in df.columns[2:]:
            brigade_lib = col
            count = row[col]
            if count > 0:
                cursor.execute("INSERT OR IGNORE INTO Brigade (lib_brigade, is_pn) VALUES (?, ?)", (brigade_lib, sheet_name == 'Services PN 2012'))
                cursor.execute("SELECT id_brigade FROM Brigade WHERE lib_brigade = ?", (brigade_lib,))
                id_brigade = cursor.fetchone()
                if id_brigade:
                    id_brigade = id_brigade[0]
                    cursor.execute("INSERT OR IGNORE INTO Enregistrer (id_brigade, id_crime, annee) VALUES (?, ?, 2012)", (id_brigade, crime_index))

# Valider et fermer la connexion
conn.commit()
conn.close()

print("Base de données créée avec succès !")

Base de données créée avec succès !


In [16]:
from openpyxl import load_workbook

# Charger le fichier Excel
fichier_excel = r"C:\Users\JadSamain\OneDrive - ISTEP\Bureau\IUT\SAE_NoSQL\SAE_MigrationNoSQL\CSV_Modifies\Crimes.xlsx"
wb = load_workbook(fichier_excel)
ws = wb.active  # Utilise la première feuille active

# Récupérer les cellules fusionnées
merged_cells = ws.merged_cells.ranges

# Trouver la cellule fusionnée contenant "Libellé index \ CGD"
cell_found = False
for merged_cell in merged_cells:
    cell = ws.cell(row=merged_cell.min_row, column=merged_cell.min_col)
    if "CGD" in str(cell.value):  # Vérifier si c'est la cellule recherchée
        col_start = merged_cell.min_col
        row_start = merged_cell.min_row
        cell_found = True
        break

# Vérification si une cellule fusionnée a été trouvée
if not cell_found:
    print("Aucune cellule fusionnée contenant 'CGD' n'a été trouvée.")
else:
    # Lire les valeurs des colonnes à droite
    cgd_values = []
    col = col_start + 1  # Première colonne à droite
    
    # Lire toutes les valeurs tant qu'on ne rencontre pas une cellule vide
    while col <= ws.max_column and ws.cell(row=row_start, column=col).value:
        cgd_values.append(ws.cell(row=row_start, column=col).value)
        col += 1

    # Afficher les résultats
    print("Cellule fusionnée trouvée:", cell.value)
    print("Données à droite:", cgd_values)


Aucune cellule fusionnée contenant 'CGD' n'a été trouvée.
